In [95]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import time

In [96]:
class Mynetwork(nn.Module):
    def __init__(self,input_num=1 , out_num=1,hidden_num=128):
        super().__init__()
        self.MLP=nn.Sequential(
            nn.Linear(input_num, hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),       
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),   
            nn.Linear(hidden_num,out_num),
        )
        pass
    def forward(self,x):
        return self.MLP(x)+self.MLP(-x)
    
def potential(x):
    poten=x*torch.sin(torch.pi*x*2/3)+x**2*2
    return poten


In [97]:
exe_num=10
sigma=0.01

total_time=time.time()
file_name='f3_model_10_0.01'

en_num=40
extend_num=10
epoch=200000
lr=0.01

h_bar=1
m=1
b_lap:float=-h_bar**2/(2*m)

# 同时对于库伦势函数, 取e=1, 4\pi\epsilon_0=1, E_n=-1/(2n^2)
dtype=torch.float32
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

La=-10
Lb =10
L=Lb-La  # domain length
N = 800   # number of interior points # 对时间成本来说几乎是平方量级
h :float= L / (N+1)
grid=torch.linspace(La,Lb,N+2,dtype=dtype,device=device)
grid=grid[1:-1].unsqueeze(-1)


diag = -2.0 / h**2 * torch.ones(N,device=device) * b_lap
off_diag = 1.0 / h**2 * torch.ones(N - 1,device=device) * b_lap


In [98]:

#######————————————————————————————————————########    
eig_loss_list=[]
pre_loss_list=[]

potential_list=[]
eig_list=[]
rand_real_eig_list=[]

for execution in range(exe_num):
    
    V_diag=potential(grid)
    A = torch.diag(diag) + torch.diag(off_diag,diagonal=1) + torch.diag(off_diag, diagonal=-1)+torch.diag(V_diag.flatten())
    eigenvalues= torch.linalg.eigvalsh(A)
    real_en_0=eigenvalues[:en_num].detach()
    
    ####################
    rand_en=sigma*torch.randn(en_num,device=device,dtype=dtype)+1
    ####################
    real_en=real_en_0*rand_en
    real_en,_=torch.sort(real_en)
    
    rand_real_eig_list.append(real_en.detach().cpu().numpy())
    
    
    # torch.manual_seed(seed=42) 
    os.makedirs(f'./{file_name}', exist_ok=True)
    
    model=Mynetwork().to(device=device,dtype=dtype)
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=50,threshold=1e-4)
    loss_fn=nn.L1Loss()
    
    pre_loss=loss_fn(real_en,real_en_0)
    pre_loss_list.append(pre_loss.item())

    init_time=time.time()
    for i in range(epoch):
        optimizer.zero_grad()
        V_diag=model(grid)
        A = torch.diag(diag) + torch.diag(off_diag,diagonal=1) + torch.diag(off_diag, diagonal=-1)+torch.diag(V_diag.flatten())
        eigenvalues= torch.linalg.eigvalsh(A)
        output=eigenvalues[:en_num]
        
        loss=loss_fn(output,real_en)
        loss.backward()
        optimizer.step()
        
        scheduler.step(loss)
        if optimizer.param_groups[0]["lr"] <= 1.1e-8:break
        
    final_loss=loss.item()
    final_time=time.time()-init_time
    final_epoch=i+1
    

    eig_loss_list.append(final_loss)
    
    eig_list.append(output.detach().cpu().numpy())
    potential_list.append(V_diag.flatten().detach().cpu().numpy())
    # torch.save(model.state_dict(),f'./{file_name}/model_para.pth')

    del model
    torch.cuda.empty_cache()
    print(f'Execution {execution+1} completed.')
    print(f'time: {time.time()-init_time:.2f}s , epoch: {final_epoch} , loss: {final_loss:.4f}')
    print('total time:',(time.time()-total_time)/60,' min') 


Execution 1 completed.
time: 19.19s , epoch: 1969 , loss: 0.2775
total time: 0.320277468363444  min
Execution 2 completed.
time: 5.52s , epoch: 581 , loss: 0.4294
total time: 0.41245527664820353  min
Execution 3 completed.
time: 17.02s , epoch: 1737 , loss: 0.2800
total time: 0.6962354779243469  min
Execution 4 completed.
time: 12.46s , epoch: 1251 , loss: 0.2601
total time: 0.9039628148078919  min
Execution 5 completed.
time: 23.81s , epoch: 2401 , loss: 0.2425
total time: 1.3009965022404988  min
Execution 6 completed.
time: 27.02s , epoch: 2687 , loss: 0.2645
total time: 1.7515223026275635  min
Execution 7 completed.
time: 11.53s , epoch: 1208 , loss: 0.2768
total time: 1.9437567154566446  min
Execution 8 completed.
time: 30.45s , epoch: 3074 , loss: 0.3383
total time: 2.4513586362202964  min
Execution 9 completed.
time: 25.97s , epoch: 2654 , loss: 0.2564
total time: 2.884372576077779  min
Execution 10 completed.
time: 28.41s , epoch: 2995 , loss: 0.2843
total time: 3.35798193613688

In [99]:
real_en_0=real_en_0.cpu().numpy()
plt.plot(pre_loss_list,label='pre_rand_loss')
plt.title(f'pre_rand_eig_loss')
plt.legend()
plt.savefig(f'./{file_name}/pre_rand_eig_loss.png')
plt.clf()

plt.plot(eig_loss_list,label='eig_loss')
plt.title(f'eig_loss')
plt.legend()
plt.savefig(f'./{file_name}/eig_loss.png')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [100]:
grid=torch.linspace(La,Lb,N+2)
grid=grid[1:-1]
real_poten=potential(grid)
real_poten=real_poten.numpy()

dpi=600

for i in range(len(potential_list)):
    plt.plot(grid,potential_list[i],zorder=i+1,color='g',alpha=0.5)
plt.plot(grid,real_poten,label='real',zorder=i+10,color='r')
plt.ylim(real_poten.min()-10,real_poten.max()+10)
plt.title(f'potential band')
plt.legend()
plt.savefig(f'./{file_name}/potential_band.png',dpi=dpi)
plt.clf()

count=np.array([i for i in range(en_num)])
for i in range(len(eig_list)):
    plt.plot(count,eig_list[i],zorder=i+1,color='g',alpha=0.5)
plt.plot(count,real_en_0,label='real',zorder=i+10,color='r')
plt.ylim(real_en_0.min()-1,real_en_0.max()+1)
plt.title(f'eigenvalue band')
plt.legend()
plt.savefig(f'./{file_name}/eig_band.png',dpi=dpi)
plt.clf()

for i in range(len(rand_real_eig_list)):
    plt.plot(count,rand_real_eig_list[i],zorder=i+1,color='g',alpha=0.5)
plt.plot(count,real_en_0,label='real',zorder=i+10,color='r')
plt.ylim(real_en_0.min()-1,real_en_0.max()+1)
plt.title(f'initial eigenvalue band')
plt.legend()
plt.savefig(f'./{file_name}/initial_eig_band.png',dpi=dpi)
plt.clf()

print(' 所有执行完成 ')

 所有执行完成 


<Figure size 640x480 with 0 Axes>